<a href="https://colab.research.google.com/github/aaron-ruhl/EasyVisa/blob/main/DSBA_Project_ET_EasyVisa_Fullcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EasyVisa Project

## Context:

Business communities in the United States are facing high demand for human resources, but one of the constant challenges is identifying and attracting the right talent, which is perhaps the most important element in remaining competitive. Companies in the United States look for hard-working, talented, and qualified individuals both locally as well as abroad.

The Immigration and Nationality Act (INA) of the US permits foreign workers to come to the United States to work on either a temporary or permanent basis. The act also protects US workers against adverse impacts on their wages or working conditions by ensuring US employers' compliance with statutory requirements when they hire foreign workers to fill workforce shortages. The immigration programs are administered by the Office of Foreign Labor Certification (OFLC).

OFLC processes job certification applications for employers seeking to bring foreign workers into the United States and grants certifications in those cases where employers can demonstrate that there are not sufficient US workers available to perform the work at wages that meet or exceed the wage paid for the occupation in the area of intended employment.

## Objective:

In FY 2016, the OFLC processed 775,979 employer applications for 1,699,957 positions for temporary and permanent labor certifications. This was a nine percent increase in the overall number of processed applications from the previous year. The process of reviewing every case is becoming a tedious task as the number of applicants is increasing every year.

The increasing number of applicants every year calls for a Machine Learning based solution that can help in shortlisting the candidates having higher chances of VISA approval. OFLC has hired your firm EasyVisa for data-driven solutions. You as a data scientist have to analyze the data provided and, with the help of a classification model:

* Facilitate the process of visa approvals.
* Recommend a suitable profile for the applicants for whom the visa should be certified or denied based on the drivers that significantly influence the case status.


## Data Description

The data contains the different attributes of the employee and the employer. The detailed data dictionary is given below.

* case_id: ID of each visa application
* continent: Information of continent the employee
* education_of_employee: Information of education of the employee
* has_job_experience: Does the employee has any job experience? Y= Yes; N = No
* requires_job_training: Does the employee require any job training? Y = Yes; N = No
* no_of_employees: Number of employees in the employer's company
* yr_of_estab: Year in which the employer's company was established
* region_of_employment: Information of foreign worker's intended region of employment in the US.
* prevailing_wage:  Average wage paid to similarly employed workers in a specific occupation in the area of intended employment. The purpose of the prevailing wage is to ensure that the foreign worker is not underpaid compared to other workers offering the same or similar service in the same area of employment.
* unit_of_wage: Unit of prevailing wage. Values include Hourly, Weekly, Monthly, and Yearly.
* full_time_position: Is the position of work full-time? Y = Full Time Position; N = Part Time Position
* case_status:  Flag indicating if the Visa was certified or denied

## Importing necessary libraries and data

In [ ]:
# Reading and manipulating dataframes
import numpy as np
import pandas as pd

# Visualizations of data
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Using Google Colab, I first uploaded the file
from google.colab import files
uploaded = files.upload()

In [ ]:
# Then used pandas to read the file with the help of 'io'
import io
data = pd.read_csv(io.BytesIO(uploaded['EasyVisa.csv']))

## Data Overview

- Observations
- Sanity checks

**Lets get a quick look at the data**

In [ ]:
data.head()

In [ ]:
data.tail()

**What is the shape of the data?**

In [ ]:
data.shape

- 25480 observations, with only 12 columns. Could be result of trimming off excessive features. Either way it is a small amount which I think will help with the time required to build models.

**What about dtypes and non-null counts?**

In [ ]:
data.info()

- Mostly categorical, with a few numerical columns
- Non-null counts are all clean

**Overview of numerical columns**

In [ ]:
data.describe(include=np.number).T

`no_of_employees`
  - Negative values like the minimum(-26) should be investigated further.
  - Most of the values centered around 2100
  - Massive rightward skew, mean of 5667 with a std of 22877.

`yr_of_estab`
  - Centered around 1990-2000, but the left tail was highly skewed.
  - Mean was around 1976 with a std of 42.
  - Older businesses have a harder time getting approved?

`prevailing_wage`
  - Immense range of values from 2.14 to 319210.27
  - Most of the values were centered around 70000
  - Mean of 74455.8 and std of 52816

**Overview of categorical columns(plus binary)**

In [ ]:
data.describe(include="object").T

Observations:

`Case_id`:
  - This is just a unique variable that could be dropped for this situation.

`Continent`:
  - Mostly "Asia" with around eight thousand spread across the other 5 categories.

`Education_of_employee`:
  - Might work as an ordinal variable, with education increasing at each of the 4 levels.
    - Basically, they stack because to get a masters one must first get the first two. so technically it is the "third" level.
    - Also reducing the overall amount of features that I feed into the model building process seems like a prudent choice and I have seen this particular category done this way before.

`Has_job_experience`:
  - Seems it was around 50/50 "Y" or "N".

`Requires_job_training`:
  - Most of the responses were "N", only around 3000 "Y" out of 25480.

`Region_of_employment`:
- Does not seem like any of the classes dominated this feature.
- Five different regions.

`Unit_of_wage`:
  - Year was a highly frequent occurence and might overpower the minority classes in this feature.

`Full_time_position`:
  - Skewed with only 2 classes like 'requires_job_training', with "Y" as the most frequent.

`Case_status`:
  - Split 1/3 "Denied" and 2/3 "Certified".
    - "Denied" is the minority class.

OVERALL:
  - Exceptionally low cardinality in all of these variables. Rare classes will be my primary concern.

**Quick isnull() check**

In [ ]:
data.isnull().mean()

**Quick duplicate checks**

In [ ]:
len(data.case_id.unique())

In [ ]:
data.duplicated().sum()

## Exploratory Data Analysis (EDA)

In [ ]:
# This function just works very well and makes great looking graphs so I decided to use it again.


def stacked_barplot(data, predictor, target):
    """
    Print the category counts and plot a stacked bar chart

    data: dataframe
    predictor: independent variable
    target: target variable
    """
    count = data[predictor].nunique()
    sorter = data[target].value_counts().index[-1]
    tab1 = pd.crosstab(data[predictor], data[target], margins=True).sort_values(
        by=sorter, ascending=False
    )
    print("-" * 120)
    print(tab1)

    tab = pd.crosstab(data[predictor], data[target], normalize="index").sort_values(
        by=sorter, ascending=False
    )
    tab.plot(kind="bar", stacked=True, figsize=(count + 1, 5))
    plt.legend(
        loc="lower left",
        frameon=False,
    )
    plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
    plt.show()

**Leading Questions**:


1. Those with higher education may want to travel abroad for a well-paid job. Does education play a role in Visa certification?



In [ ]:
stacked_barplot(data,'education_of_employee','case_status')

- Yes, it appears to be important according to this data. Appears more likely to get a visa if one has a higher level of education.

2. How does the visa status vary across different continents?


In [ ]:
stacked_barplot(data,'continent','case_status')

- Changes across different continents but the range is overall pretty small.


3. Experienced professionals might look abroad for opportunities to improve their lifestyles and career development. Does work experience influence visa status?




In [ ]:
stacked_barplot(data,'has_job_experience','case_status')

- Appears to indicate that applicants with work experience were accepted more often on average.

4. In the United States, employees are paid at different intervals. Which pay unit is most likely to be certified for a visa?


In [ ]:
stacked_barplot(data,'unit_of_wage','case_status')

- Yearly is the most likely, but also the vast majority of the cases.

5. The US government has established a prevailing wage to protect local talent and foreign workers. How does the visa status change with the prevailing wage?

In [ ]:
# Lets plot the distribution using matplotlib since it is continuous
fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(111)

data[data['case_status']=="Denied"]['prevailing_wage'].hist(bins=50, ax=ax, color='red')
data[data['case_status']=="Certified"]['prevailing_wage'].hist(bins=50, ax=ax, color='green', alpha=0.5)

ax.set_title('prevailing_wage')
ax.set_xlabel('prevailing_wage')
ax.legend(['Denied','Certified'], loc='best')

- Not any major changes except for the lower end of the variable, I explore this further in EDA below.

**before splitting, lets get a further look at the data and try to see what else is going on**

### Categorical Univariate Analysis

**`education_of_employee`**

In [ ]:
data.education_of_employee.unique()

Education of employee recorded in this dataset could be considered ordinal.
<br></br>For example:
<br>  'education_of_employee'= {["High School": 1, "Bachelor's":2, "Masters":3, "Doctorate":4]} </br>

In [ ]:
# Let's make a histogram to get familiar with the
# variable distribution.

fig = data['education_of_employee'].hist(bins=60)

fig.set_title('Education of Employee')
fig.set_xlabel('Unique Levels of Education')
fig.set_ylabel('Number of Applicants')

Mostly within the mid-range: "Bachelor's" <--> "Master's"

In [ ]:
data.education_of_employee.value_counts(normalize=True)

"Doctorate" is a rare class, but it is almost 9% of data. Not exceedingly rare.

**`continent`**

In [ ]:
# Let's see what unique categories we have.
data.continent.unique()

Decent diversity in this feature.

In [ ]:
# Let's make a histogram to get familiar with the variable distribution.

fig = data['continent'].hist(bins=40)

fig.set_title('Continent')
fig.set_xlabel('Origin-Continent of Applicants')
fig.set_ylabel('No. of Applicants')

As the code below confirms, "South America", "Africa", and "Oceania" were substantially more rare than "Asia".
<br>"North America" and "Europe" were more common, but still much less prevalent in the data than "Asia".</br>         

In [ ]:
# Trying to determine the prevalence of each category
data.continent.value_counts(normalize=True)

**`region_of_employment`**

In [ ]:
# Let's make a histogram to get familiar with the variable distribution.

fig = data.region_of_employment.hist(bins=50)

fig.set_title("Region Of Employement")
fig.set_xlabel("Regions that these Applicants Prefered")
fig.set_ylabel("No. of Applicants")

In [ ]:
# Trying to determine the prevalence of each category
data.region_of_employment.value_counts(normalize=True)

"Island" is a rare minority of the cases.

**`unit_of_wage`**

In [ ]:
# Let's make a histogram to get familiar with the variable distribution.

fig = data.unit_of_wage.hist(bins=50)

fig.set_title("")
fig.set_xlabel("")
fig.set_ylabel("No. of Applicants")

In [ ]:
# Trying to determine the prevalence of each category
data.unit_of_wage.value_counts(normalize=True)

Year is the overwhelming majority of cases.

**Okay! Now lets look at the numerical columns**

### Numerical Univariate Analysis

In [ ]:
# I decided to try and do things differently this time (transform) and found this function elsewhere and thought it worked really well for this purpose
import scipy.stats as stats

# Function to create a histogram, a Q-Q plot and
# a boxplot.


def diagnostic_plots(df, variable, bins=28):
    # The function takes a dataframe (df) and
    # the variable of interest as arguments.

    # Define figure size.
    plt.figure(figsize=(16, 4))

    # histogram
    plt.subplot(1, 3, 1)
    sns.histplot(df[variable], bins=bins)
    plt.title('Histogram')

    # Q-Q plot
    plt.subplot(1, 3, 2)
    stats.probplot(df[variable], dist="norm", plot=plt)
    plt.ylabel('Case Status')

    # boxplot
    plt.subplot(1, 3, 3)
    sns.boxplot(y=df[variable])
    plt.title('Boxplot')

    plt.show()

In [ ]:
# make a list of the numerical columns and iterate through that list to create a quick and informative display

num_cols = ['no_of_employees','yr_of_estab','prevailing_wage']

for specific_col in num_cols:
    print('\n')
    diagnostic_plots(data, specific_col)
    print('\n')

`no_of_employees`
  - As mentioned above, mostly centered around 2000. Unfortunately this means that a massive amount of data was an outlier.
  - Massive range could cause issues with model building time.

`yr_of_estab`
  - Some sort of grouping might be appropriate here, but I have not been properly introduced to such a technique.
    - It certainly will not prevent me from finishing the requirements of this project in its current form. I focused on transforming the other two continuous variables.

`prevailing_wage`
  - Not bad, but there was a large amount of outliers and the lower end seems very skewed. Basically looks like a bimodal distribution.
  - Massive range could cause issues with model building time.

**Let's Check out those negative values for 'no_of_employees'**

In [ ]:
print(len(data[data['no_of_employees']<=0]),"rows with less than zero employees")
#data[data['no_of_employees']<=0]

- Ultimate solution could be imputing these values and adding a missing indicator for each of these rows because they do appear to be pretty normal otherwise. This would technically be adding an additional column for only 30 observations! They better be significant!

- Anyways I decided, because we have not learned that yet and I still have nightmares from the hackathon, to just remove these rows for now.

- Also, one could say that removing 33 potentially erroneous observations is not very likely to add significant bias to predictions made using a set of 25480 observations. Not impossible, but very unlikely.

### Bivariate Analysis

**Now, I display all of the categorical/binary variables 'case_status', split into different classes of each feature.**

In [ ]:
# Making a list that I call 'cat_cols'
cat_cols = ['education_of_employee','continent','region_of_employment','unit_of_wage','has_job_experience','requires_job_training','full_time_position']

# Initializing total_applicants as the length of the data (25480)
total_applicants = len(data)

# Iterating through cat_cols, running stacked_barplots and a custom graph to show category frequency!
for col in cat_cols:

    # Defined above
    stacked_barplot(data, col, 'case_status')

    # Let's plot the category frequency.
    # That is, the percentage of applicants with each label.

    temp_df = pd.Series(data[col].value_counts() / total_applicants)

    # Make plot with these percentages.
    fig = temp_df.sort_values(ascending=False).plot.bar()
    fig.set_xlabel(col)

    # Add a line at 5 % to flag the threshold for rare categories.
    fig.axhline(y=0.05, color='red')
    fig.set_ylabel('Percentage of Applicants')
    plt.show()

Observations:
- `Education_of_employee` adds an excellent amount of predictive value.  
  - "Doctorate" applicants were denied visa status in around 15% of the cases; "Highschool" applicants were denied in approx. 65% of cases.
  - Also the amount of data in all categories is above 5% so no rare cases and low cardinality aswell.
- `Continent` adds decent predictive value, but the categories will need to be handled properly.
  - Asia was the origin continent for the vast majority of cases. The noise in the data is largely dominated by applicants from Asia with a few rare classes.
    - There is definitely some predictive value for "Africa" even though it is rare, I will try to keep this category seperate in train/test.
    - "Oceania" & "South America" should be combined into an "Other" column.
      - Rare cases often cause a mismatch between training and test.
      - Little difference in proportion of 'case_status' between these two continents.
- `region_of_employement` proportion of 'case_status' was not constant across all classes, but not a very large swing in overall proportion of 'case_status'.
  - "Island" might cause some issues with building train/test, but I want to try and keep it seperated.
- `unit_of_wage` appears to offer an excellent predictive value, but I will need to handle the classes carefully.
  - "Week" & "Month" are rare classes in this data. I will combine them to improve model building performance.
    - "Month" has only 89 entries so it has a very high chance of causing issues building train/test.
    -  Exact same proportion of 'case_status' between these two classes. So seperated didnt no really seem to offer alot of value.
    - Also the idea of combining monthly and weekly paid applicants works out because "technically" they are collectively in-between "Year" and "Hour".


**Excellent, now let's take a look at the numerical columns**

In [ ]:
num_cols=['no_of_employees','yr_of_estab','prevailing_wage']

for specific_col in num_cols:
    fig = plt.figure(figsize=(12,9))
    ax = fig.add_subplot(111)

    data[data['case_status']=="Denied"][specific_col].hist(bins=50, ax=ax, color='red')
    data[data['case_status']=="Certified"][specific_col].hist(bins=50, ax=ax, color='green', alpha=0.5)

    ax.set_title(specific_col)
    ax.set_xlabel(specific_col)
    ax.legend(['Denied','Certified'], loc='best')

`no_of_employees`:
  - based on this graph, this variable seems pretty useless, but there could be alot of useful variation within the lower end of the range.

`yr_of_estab`:
  - Appears to have a relatively constant frequency of 'case_status'. Perhaps if it was discreetly binned it would give more information. I left this out for simplicity. Also as I mentioned it does not seem all that significant overall.

`prevailing_wage`:
  - Seems like a significant increase in proportion of "Denied" 'case_status' was recorded within the lower end of this feature.

**Let's look at the tail end of 'prevailing_wage' & 'no_of_employees'**

In [ ]:
# Starting with 'prevailing_wage', looking at the lower end

col_temp = 'prevailing_wage'
fig = plt.figure()
ax = fig.add_subplot(111)

data[data['case_status']=="Denied"][col_temp].hist(bins=1000, ax=ax, color='red')
data[data['case_status']=="Certified"][col_temp].hist(bins=1000, ax=ax, color='green', alpha=0.8)

ax.set_xlabel(col_temp)
ax.set_xlim(0,10000) # What I refer to as the "lower end" (0,10000)
ax.set_xlabel(col_temp)
ax.legend(['Denied','Certified'], loc='best')

- Rapidly shifts from being mostly "Denied" around 1000 and the proportion onwards is mostly constant.

In [ ]:
col_temp = 'prevailing_wage'

fig = plt.figure()
ax = fig.add_subplot(111)

data[data['case_status']=="Denied"][col_temp].hist(bins=1000, ax=ax, color='red')
data[data['case_status']=="Certified"][col_temp].hist(bins=1000, ax=ax, color='green', alpha=0.8)

ax.set_xlabel(col_temp)
ax.set_xlim(150000,data.prevailing_wage.max())
ax.set_ylim(0,25)
ax.set_xlabel(col_temp)
ax.legend(['Denied','Certified'], loc='best')

- Alot of noise in the upper end of this feature.

In [ ]:
data[data['prevailing_wage']<=900].unit_of_wage.value_counts()

- Almost the entire "Hourly" segment was within this range of equal to or less than 900. Okay, perhaps confirming that hourly workers are typically paid less overall.

**Now Let's look at 'no_of_employees'**

In [ ]:
col_temp = 'no_of_employees'
fig = plt.figure()
ax = fig.add_subplot(111)

data[data['case_status']=="Denied"][col_temp].hist(bins=1000, ax=ax, color='red')
data[data['case_status']=="Certified"][col_temp].hist(bins=1000, ax=ax, color='green', alpha=0.8)

ax.set_xlabel(col_temp)
ax.set_xlim(0,10000)
ax.set_xlabel(col_temp)
ax.legend(['Denied','Certified'], loc='best')

- Appears to be relatively constant

In [ ]:
col_temp = 'no_of_employees'

fig = plt.figure()
ax = fig.add_subplot(111)

data[data['case_status']=="Denied"][col_temp].hist(bins=1000, ax=ax, color='red')
data[data['case_status']=="Certified"][col_temp].hist(bins=1000, ax=ax, color='green', alpha=0.8)

ax.set_xlabel(col_temp)
ax.set_xlim(25000,150000)
ax.set_ylim(0,25)
ax.set_xlabel(col_temp)
ax.legend(['Denied','Certified'], loc='best')

- Also alot of noise in the upper end of this feature.

In [ ]:
data[data['no_of_employees']<=7500].region_of_employment.value_counts()

- Includes almost all of the Island category. I guess Islands do not have alot of people. Good to see that the Island category appears normal in this respect because it is so rare within the data.

**Let's look at the Correlation Matrix**

In [ ]:
sns.heatmap(data.corr(),annot=True,linewidths=.5,center=0,cbar=False,cmap="Spectral")
plt.show()

- Not much to say here, besides no correlation detected. Simply just not alot of opportunities for this to occur within this set of features. They seem to be very independent variables.

**Awesome, time to move on to data preprocessing**

## Data Preprocessing

**Let's convert the objects into categories**

In [ ]:
for feature in data.columns: # Loop through all columns in the dataframe
    if data[feature].dtype == 'object': # Only apply for columns with categorical strings
        data[feature] = pd.Categorical(data[feature])# Replace strings with an integer
data.head(10)

**Remove the 33 rows mentioned above as I initialize a new df to create/prepare X and y**

In [ ]:
# Initialize df as the entire dataframe minus the 33 rows mentioned above with negative values for 'no_of_employees' I went with CCA for simplicity, it was mentioned in EDA above

df = data.drop(data[data.no_of_employees<0].index) #Using the returned indices to drop rows with less than zero employees

# Splitting into X and y values seperately so I can keep there original values
X = df.drop(['case_status', 'case_id'], axis=1)
y = df.pop('case_status')

### Feature Engineering and Preparing data for Modeling

In [ ]:
# Making a dictionary to perform the replacements I justified above in EDA and other necessary changes such as binary strings

replaceStruct = {
                "education_of_employee": {"High School":1,"Bachelor's":2,"Master's":3,"Doctorate":4},
                "unit_of_wage":     {"Week": "Week_or_Month", "Month": "Week_or_Month"},
                "continent":     {"South America": "Other", "Oceania": "Other"},
                "requires_job_training":     {"N": 0, "Y": 1 },
                "has_job_experience":     {"N": 0, "Y": 1 },
                "full_time_position":     {"N": 0, "Y": 1 }
                    }

# These will be the columns I want to pass into oneHotCols, basically so I am absolutely sure what the function below is doing
oneHotCols=["unit_of_wage","continent","region_of_employment"]

In [ ]:
# Here I make replacements and initialize it as X_main to prevent altering the original X, save a copy before doing oneHotCols as X_eda, then finally get_dummies for X_main

X_main=X.replace(replaceStruct)
X_eda = X_main.copy()

X_main=pd.get_dummies(X_main, columns=oneHotCols)

**Splitting the data into Simple and Transformed sets**

In [ ]:
# importing train_test_split
from sklearn.model_selection import train_test_split

# Okay I hope this is not a terribly bad way of doing this, but all I am doing is making a "simple(s)" and "transformed(t)" train_test_split.
# Later I also make one for Logistic Regression by using the unaltered X
# The third X_train_EDA is simply to make graphs of the replacements I made to see what happened, that is why there is _,_,_ after it. I just do not need those variables.

X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_main, y, test_size=.30, random_state=1, stratify=y)

X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(X_main, y, test_size=.30, random_state=1, stratify=y)

X_train_EDA, _,_,_= train_test_split(X_eda, y, test_size=.30, random_state=1, stratify=y)

# Printing out a sanity check, which helps me as a beginner to see if it was working properly

print("Perfect match between different train/test sets is: ", all(X_train_s == X_train_t),
      "\n\nShape of simple(s) train/test: \n",X_train_s.shape, X_test_s.shape,
      "\n\nShape of transformed(t) train/test: \n", X_train_t.shape, X_test_t.shape,
      "\n\nProportion of case_status:\n", y_train_s.value_counts(normalize=True),sep='')

In [ ]:
# For EDA purposes I am going to stick back on the 'case_status'. Essentially I just want to make cool graphs with the stacked barplots

X_train_EDA = X_train_EDA.join(y_train_s)
#X_train_EDA

In [ ]:
# Quick sanity check that helps when rerunning multiple times

print(len(X_train_s[X_train_s.no_of_employees<0]),"rows with no_of_employees recorded as less than zero.")

**Setting the 'case_status' to the desired binary variables**

In [ ]:
y_train_s = y_train_s.replace({'Certified':0,'Denied':1})
y_train_t = y_train_t.replace({'Certified':0,'Denied':1})

In [ ]:
y_test_s = y_test_s.replace({'Certified':0,'Denied':1})
y_test_t = y_test_t.replace({'Certified':0,'Denied':1})

- I made the minority class the positive case.

### Treating Outliers

**Let's try using a log transform on these two columns**

In [ ]:
# same graph defined above for EDA

num_cols = ['no_of_employees','prevailing_wage']

for specific_col in num_cols:
    print('\n')
    X_train_t[specific_col + '_log'] = np.log(X_train_t[specific_col])   # this is the tricky bit, I am making a new column by using the variable name plus '_log' and then calling it in diagnostic_plots()
    diagnostic_plots(X_train_t, specific_col+'_log',bins=28)
    print('\n')

- Wow, it looks like no_of_employees cleaned up pretty well with a log transform, but still some outliers.
- 'prevailing_wage' still is pretty skewed

**Now let's try using a squre root transform**

In [ ]:
# Same as above, except now the transformation is "**(1/2)" or square root

num_cols = ['no_of_employees','prevailing_wage']

for specific_col in num_cols:
    print('\n')
    X_train_t[specific_col + '_sqrt'] = (X_train_t[specific_col])**(1/2)
    diagnostic_plots(X_train_t, specific_col+'_sqrt',bins=28)
    print('\n')

'prevailing_wage' contains almost no outliers with this transformation.

**Okay, I think I know which transforms to apply to both of these columns.**

In [ ]:
# Great, so I will do a log transform on 'no_of_employees'
# And a square root transform on 'prevailing_wage'

X_train_t.drop(['no_of_employees','no_of_employees_sqrt','prevailing_wage','prevailing_wage_log'],axis=1,inplace=True) # Above I was literally creating these columns so now I selectively remove the ones I no longer need

**Do not Forget to apply same thing to the X_test!**

In [ ]:
# making the new columns
X_test_t['no_of_employees_log']=np.log(X_test_t['no_of_employees'])
X_test_t['prevailing_wage_sqrt']=(X_test_t['prevailing_wage'])**(1/2)

# dropping the old columns
X_test_t.drop(['prevailing_wage','no_of_employees'],axis=1,inplace=True)

- Notice that I made the "simple" X_train_s earlier because I am new to the concept of transforming variables. I just feel more comfortable with being able to double check...even if it was a pain to code it all.

## EDA After Splitting and Modifications

- It is a good idea to explore the data once again after manipulating it.

In [ ]:
X_train_s.shape, X_test_s.shape

In [ ]:
X_train_t.shape, X_test_t.shape

### Categorical Comparison

In [ ]:
cat_cols = ['education_of_employee','continent','region_of_employment','unit_of_wage','has_job_experience','requires_job_training','full_time_position']

# Let's plot the category frequency.
# That is, the percentage of houses with each label.

total_applicants = len(X_train_EDA)

for specific_col in cat_cols:
    print('\n'+specific_col+':')

    stacked_barplot(X_train_EDA, specific_col,'case_status')

    temp_df = pd.Series(X_train_EDA[specific_col].value_counts() / total_applicants)

    # Make plot with these percentages.
    fig = temp_df.sort_values(ascending=False).plot.bar()

    # Add a line at 5 % to flag the threshold for rare categories.
    fig.axhline(y=0.05, color='red')
    fig.set_ylabel('Percentage of Applicants')
    plt.show()

**X_train is capturing similar information as seen in original data even after those replacements**

### Numerical Comparison

In [ ]:
# Let's plot the 'no_of_employees' relationship with 'case_status' in X_train_s

col_temp = 'no_of_employees'

fig = plt.figure()
ax = fig.add_subplot(111)

X_train_EDA[X_train_EDA['case_status']=="Denied"][col_temp].hist(bins=28, ax=ax, color='red')
X_train_EDA[X_train_EDA['case_status']=="Certified"][col_temp].hist(bins=28, ax=ax, color='green', alpha=0.8)

ax.set_title(col_temp)
ax.set_xlabel(col_temp)
ax.legend(['X_train_s - Denied','X_train_s - Certified'], loc='best');

In [ ]:
# Let's plot the 'no_of_employees_log' relationship with 'case_status' as it would be in X_train_t

col_temp = 'no_of_employees'

fig = plt.figure()
ax = fig.add_subplot(111)

X_train_EDA[col_temp+'_log'] = np.log(X_train_EDA[col_temp])

X_train_EDA[X_train_EDA['case_status']=="Denied"][col_temp+'_log'].hist(bins=28, ax=ax, color='red')
X_train_EDA[X_train_EDA['case_status']=="Certified"][col_temp+'_log'].hist(bins=28, ax=ax, color='green', alpha=0.8)

ax.set_title(col_temp+'_log')
ax.set_xlabel(col_temp+'_log')
ax.legend(['X_train_t - Denied','X_train_t - Certified'], loc='best');

**Similar to original distribution for X_train_s and the transformation could  be helping for X_train_t. Significant predictive value is not very clear from either graph and it almost looks like a relatively constant proportion of 'case_status'**

In [ ]:
# Let's plot the 'prevailing_wage' relationship with 'case_status' in X_train_s

col_temp = 'prevailing_wage'

fig = plt.figure()
ax = fig.add_subplot(111)

X_train_EDA[X_train_EDA['case_status']=="Denied"][col_temp].hist(bins=50, ax=ax, color='red')
X_train_EDA[X_train_EDA['case_status']=="Certified"][col_temp].hist(bins=50, ax=ax, color='green', alpha=0.8)

ax.set_title(col_temp)
ax.set_xlabel(col_temp)
ax.legend(['X_train_s - Denied','X_train_s - Certified'], loc='best');

In [ ]:
# Let's plot the 'prevailing_wage_sqrt' relationship with 'case_status' in X_train_t

col_temp = 'prevailing_wage'

fig = plt.figure()
ax = fig.add_subplot(111)

X_train_EDA[col_temp+'_sqrt'] = X_train_EDA[col_temp]**(1/2)

X_train_EDA[X_train_EDA['case_status']=="Denied"][col_temp+'_sqrt'].hist(bins=50, ax=ax, color='red')
X_train_EDA[X_train_EDA['case_status']=="Certified"][col_temp+'_sqrt'].hist(bins=50, ax=ax, color='green', alpha=0.8)

ax.set_title(col_temp+'_sqrt')
ax.set_xlabel(col_temp+'_sqrt')
ax.legend(['X_train_t - Denied','X_train_t - Certified'], loc='best');

**Also looks very similar to the original data and the trasformation might help the models "capture" all the information within the lower end of this distribution.
Appears to have almost three seperate clusters. High, medium, and low. Unfortunately, I was unsure how to use that information.**

In [ ]:
col_temp = 'yr_of_estab'

fig = plt.figure()
ax = fig.add_subplot(111)

data[col_temp].hist(bins=30, ax=ax, color='yellow')
X_train_s[col_temp].hist(bins=30, ax=ax, color='rebeccaPurple', alpha=0.8)

ax.set_title(col_temp)
ax.set_xlabel(col_temp)
ax.legend(['Data','X_train_s'], loc='best');

In [ ]:
sns.heatmap(X_train_s.loc[:,['no_of_employees','prevailing_wage','yr_of_estab']].corr(),annot=True,linewidths=.5,center=0,cbar=False,cmap="Spectral")
plt.title('Correlation Matrix')
plt.show()

In [ ]:
sns.heatmap(X_train_t.loc[:,['no_of_employees_log','prevailing_wage_sqrt','yr_of_estab']].corr(),annot=True,linewidths=.5,center=0,cbar=False,cmap="Spectral")
plt.title('Correlation Matrix')
plt.show()

**Not to spend too long here, this all looks great! Now it is time to start building some models**

## Model Building

In [ ]:
## Function to create confusion matrix that was borrowed from Great Learning because it worked well here
def make_confusion_matrix(model,y_actual,X_test,labels=[0, 1]):
    '''
    model : classifier to predict values of X
    y_actual : ground truth

    '''
    y_predict = model.predict(X_test)

    cm=metrics.confusion_matrix(y_actual, y_predict, labels=[0, 1])
    df_cm = pd.DataFrame(cm, index = [i for i in ["Actual - No","Actual - Yes"]],
                  columns = [i for i in ['Predicted - No','Predicted - Yes']])
    group_counts = ["{0:0.0f}".format(value) for value in
                cm.flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in
                         cm.flatten()/np.sum(cm)]
    labels = [f"{v1}\n{v2}" for v1, v2 in
              zip(group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
    plt.figure(figsize = (10,7))
    sns.heatmap(df_cm, annot=labels,fmt='')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
from sklearn import metrics

##  Function to calculate different metric scores of the model - Accuracy, Recall and Precision that was also borrowed from Great Learning
def get_metrics_score(model, X_train, X_test, y_train, y_test,flag=True):
    '''
    model : classifier to predict values of X

    '''
    # defining an empty list to store train and test results
    score_list=[]
    pred_train = model.predict(X_train)
    pred_test = model.predict(X_test)

    train_acc = model.score(X_train,y_train)
    test_acc = model.score(X_test,y_test)

    train_recall = metrics.recall_score(y_train,pred_train)
    test_recall = metrics.recall_score(y_test,pred_test)

    train_precision = metrics.precision_score(y_train,pred_train)
    test_precision = metrics.precision_score(y_test,pred_test)

    train_f1 = metrics.f1_score(y_train,pred_train)
    test_f1 = metrics.f1_score(y_test,pred_test)

    score_list.extend((train_acc,test_acc,train_recall,test_recall,train_precision,test_precision,train_f1,test_f1))

    # If the flag is set to True then only the following print statements will be dispayed. The default value is set to True.
    if flag == True:
        print("Accuracy on training set : ",model.score(X_train,y_train))
        print("Accuracy on test set : ",model.score(X_test,y_test))
        print("Recall on training set : ",metrics.recall_score(y_train,pred_train))
        print("Recall on test set : ",metrics.recall_score(y_test,pred_test))
        print("Precision on training set : ",metrics.precision_score(y_train,pred_train))
        print("Precision on test set : ",metrics.precision_score(y_test,pred_test))
        print("F1 on training set : ",metrics.f1_score(y_train,pred_train))
        print("F1 on test set : ",metrics.f1_score(y_test,pred_test))

    return pd.DataFrame(score_list) # returning the list with train and test scores...as a dataframe to help with comparing models later.

### Logistic Regression

- Building this model for extra practice and doing so helps give me additional context for analysis. Plus it does not take a long time to train this model.
- Same goes for the KNN model.

**Making a seperate train/test for logistic regression(requires a constant column and drop_first=True)**

In [ ]:
#Import libraries I will need for LogisticRegression model
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant

X_log=X.replace(replaceStruct)
X_log=add_constant(X_log)
X_log=pd.get_dummies(X_log, columns=oneHotCols,drop_first=True)

In [ ]:
# splitting the data
X_train_log,X_test_log,y_train_log,y_test_log=train_test_split(X_log, y, test_size=.30, random_state=1, stratify=y)

In [ ]:
# making sure we have the right dtype for our target and train
y_train_log = y_train_log.replace({'Certified':0,'Denied':1})
y_test_log = y_test_log.replace({'Certified':0,'Denied':1})

In [ ]:
X_train_log.info()

**Building the Logistic Regression Model**

In [ ]:
# fitting the model on training set
logit = sm.Logit(y_train_log, X_train_log.astype(float))
lg_s = logit.fit()

In [ ]:
# let's print the logistic regression summary
print(lg_s.summary())

**Let's check the Multicolinearity**

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# let's check the VIF of the predictors
vif_series = pd.Series(
    [variance_inflation_factor(X_train_log.values, i) for i in range(X_train_log.shape[1])],
    index=X_train_log.columns,
    dtype=float,
)
print("VIF values: \n\n{}\n".format(vif_series))

**Looks good, no major problems here. This is expected because linearly increasing variables were not correlated**

**Dropping insignificant Columns**

In [ ]:
# getting accuracy before dropping some insignificant columns

pred_train_logModel_s = lg_s.predict(X_train_log) > 0.5
pred_train_logModel_s = np.round(pred_train_logModel_s)

print("Accuracy on training set : ", metrics.accuracy_score(y_train_log, pred_train_logModel_s))

In [ ]:
X_train_log_drop1 = X_train_log.drop("yr_of_estab", axis=1)
X_test_log_drop1 = X_test_log.drop("yr_of_estab", axis=1)

In [ ]:
# fitting the model on training set
logit_drop1 = sm.Logit(y_train_log, X_train_log_drop1.astype(float))
lg_drop1 = logit_drop1.fit()

pred_train_log_drop1 = lg_drop1.predict(X_train_log_drop1)
pred_train_log_drop1 = np.round(pred_train_log_drop1)


print("Accuracy on training set : ", metrics.accuracy_score(y_train_log, pred_train_log_drop1))
print(lg_drop1.summary())

In [ ]:
X_train_log_drop2 = X_train_log_drop1.drop("no_of_employees", axis=1)
X_test_log_drop2 = X_test_log_drop1.drop("no_of_employees", axis=1)

In [ ]:
# fitting the model on training set
logit_drop2 = sm.Logit(y_train_log, X_train_log_drop2.astype(float))
lg_drop2 = logit_drop2.fit()

pred_train_log_drop2 = lg_drop2.predict(X_train_log_drop2)
pred_train_log_drop2 = np.round(pred_train_log_drop2)


print("Accuracy on training set : ", metrics.accuracy_score(y_train_log, pred_train_log_drop2))
print(lg_drop2.summary())

In [ ]:
X_train_log_drop3 = X_train_log_drop2.drop("prevailing_wage", axis=1)
X_test_log_drop3 = X_test_log_drop2.drop("prevailing_wage", axis=1)

In [ ]:
# fitting the model on training set
logit_drop3 = sm.Logit(y_train_log, X_train_log_drop3.astype(float))
lg_drop3 = logit_drop3.fit()

pred_train_log_drop3 = lg_drop3.predict(X_train_log_drop3)
pred_train_log_drop3 = np.round(pred_train_log_drop3)


print("Accuracy on training set : ", metrics.accuracy_score(y_train_log, pred_train_log_drop3))
print(lg_drop3.summary())

In [ ]:
X_train_log_drop4 = X_train_log_drop3.drop("region_of_employment_Northeast", axis=1)
X_test_log_drop4 = X_test_log_drop3.drop("region_of_employment_Northeast", axis=1)

In [ ]:
# fitting the model on training set
logit_drop4 = sm.Logit(y_train_log, X_train_log_drop4.astype(float))
lg_drop4 = logit_drop4.fit()

pred_train_log_drop4 = lg_drop4.predict(X_train_log_drop4)
pred_train_log_drop4 = np.round(pred_train_log_drop4)


print("Accuracy on training set : ", metrics.accuracy_score(y_train_log, pred_train_log_drop4))
print(lg_drop4.summary())

In [ ]:
X_train_log_drop5 = X_train_log_drop4.drop("continent_Asia", axis=1)
X_test_log_drop5 = X_test_log_drop4.drop("continent_Asia", axis=1)

In [ ]:
# fitting the model on training set
logit_drop5 = sm.Logit(y_train_log, X_train_log_drop5.astype(float))
lg_drop5 = logit_drop5.fit()

pred_train_log_drop5 = lg_drop5.predict(X_train_log_drop5)
pred_train_log_drop5 = np.round(pred_train_log_drop5)


print("Accuracy on training set : ", metrics.accuracy_score(y_train_log, pred_train_log_drop5))
print(lg_drop5.summary())

In [ ]:
X_train_log_drop6 = X_train_log_drop5.drop("continent_North America", axis=1)
X_test_log_drop6 = X_test_log_drop5.drop("continent_North America", axis=1)

In [ ]:
# fitting the model on training set
logit_drop6 = sm.Logit(y_train_log, X_train_log_drop6.astype(float))
lg_drop6 = logit_drop6.fit()

pred_train_log_drop6 = lg_drop6.predict(X_train_log_drop6)
pred_train_log_drop6 = np.round(pred_train_log_drop6)


print("Accuracy on training set : ", metrics.accuracy_score(y_train_log, pred_train_log_drop6))
print(lg_drop6.summary())

**Now we can check performance**

In [ ]:
# defining a function to compute different metrics to check performance of a classification model built using statsmodels
def model_performance_classification_statsmodels(
    model, predictors, target, threshold=0.5
):
    """
    Function to compute different metrics to check classification model performance

    model: classifier
    predictors: independent variables
    target: dependent variable
    threshold: threshold for classifying the observation as class 1
    """

    # checking which probabilities are greater than threshold
    pred_temp = model.predict(predictors) > threshold
    # rounding off the above values to get classes
    pred = np.round(pred_temp)

    acc = metrics.accuracy_score(target, pred)  # to compute Accuracy
    recall = metrics.recall_score(target, pred)  # to compute Recall
    precision = metrics.precision_score(target, pred)  # to compute Precision
    f1 = metrics.f1_score(target, pred)  # to compute F1-score

    # creating a dataframe of metrics
    df_perf = pd.DataFrame(
        {"Accuracy": acc, "Recall": recall, "Precision": precision, "F1": f1,},
        index=[0],
    )

    return df_perf

In [ ]:
log_reg_model_train_perf = model_performance_classification_statsmodels(
    lg_drop6, X_train_log_drop6, y_train_log
)

print("Training performance:")
log_reg_model_train_perf

In [ ]:
log_reg_model_test_perf = model_performance_classification_statsmodels(
    lg_drop6, X_test_log_drop6, y_test_log
)

print("Test performance:")
log_reg_model_test_perf

In [ ]:
# predicting on training set
# threshold is 0.6577, if predicted probability is greater than 0.5 the observation will be classified as 1

pred_test_logModel = lg_drop6.predict(X_test_log_drop6) > 0.5
pred_test_logModel = np.round(pred_test_logModel)

cm = metrics.confusion_matrix(y_test_log, pred_test_logModel)
df_cm = pd.DataFrame(cm, index = [i for i in ["Actual - No","Actual - Yes"]],
              columns = [i for i in ['Predicted - No','Predicted - Yes']])
group_counts = ["{0:0.0f}".format(value) for value in
            cm.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                      cm.flatten()/np.sum(cm)]
labels = [f"{v1}\n{v2}" for v1, v2 in
          zip(group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=labels,fmt='')
plt.ylabel('True label')
plt.xlabel('Predicted label')

plt.show()

- Actually seems pretty decent already, but the minority class is being ignored. A theme I discovered throughout this project.

### KNN

In [ ]:
# importing the model
from sklearn.neighbors import KNeighborsClassifier

#building the model with the untransformed data
kNN_s=KNeighborsClassifier()
kNN_s.fit(X_train_s,y_train_s)

In [ ]:
#Using above defined function to get accuracy, recall and precision on train and test set
kNN_s_score=get_metrics_score(kNN_s,X_train_s,X_test_s, y_train_s, y_test_s) #I alteredd it to take different splits of test data for obvious reasons

In [ ]:
make_confusion_matrix(kNN_s,y_test_s,X_test_s)

- This model is performing worse than the logistic regression model with a 0.5 threshold, let's try the transformed data.

In [ ]:
kNN_t=KNeighborsClassifier()
kNN_t.fit(X_train_t,y_train_t)

In [ ]:
#Using above defined function to get accuracy, recall and precision on train and test set
kNN_t_score=get_metrics_score(kNN_t,X_train_t,X_test_t, y_train_t, y_test_t)

In [ ]:
make_confusion_matrix(kNN_t,y_test_t,X_test_t)

- This model is still not very good, but that was a significant improvement!
- Still appears to be overfitting the data.

### Decision Tree

In [ ]:
#importing the model
from sklearn.tree import DecisionTreeClassifier

#building the model
dtree_s=DecisionTreeClassifier(random_state=1)
dtree_s.fit(X_train_s,y_train_s)

In [ ]:
#Using above defined function to get accuracy, recall and precision on train and test set
dtree_s_score=get_metrics_score(dtree_s,X_train_s,X_test_s, y_train_s, y_test_s)

In [ ]:
make_confusion_matrix(dtree_s,y_test_s,X_test_s)

- Not great, overfitting considerably and the results are poor precision and recall. Let's try the transformed data.

In [ ]:
#building a tree with the transformed data now
dtree_t=DecisionTreeClassifier(random_state=1)
dtree_t.fit(X_train_t,y_train_t)

In [ ]:
#Using above defined function to get accuracy, recall and precision on train and test set
dtree_t_score=get_metrics_score(dtree_t,X_train_t,X_test_t, y_train_t, y_test_t)

In [ ]:
make_confusion_matrix(dtree_t,y_test_t,X_test_t)

- Minor improvement, still poor detection of majority and minority. Let's try it with class weights.

In [ ]:
# building a tree initialized with class weights
dtree_t_weighted=DecisionTreeClassifier(random_state=1,class_weight={0:0.3,1:0.7})
dtree_t_weighted.fit(X_train_t,y_train_t)

In [ ]:
#Using above defined function to get accuracy, recall and precision on train and test set
dtree_t_weighted_score=get_metrics_score(dtree_t_weighted,X_train_t,X_test_t, y_train_t, y_test_t)

In [ ]:
make_confusion_matrix(dtree_t_weighted,y_test_t,X_test_t)

- Overall, a minor improvement that I would say is best tree model so far. However all trees seem to be overfitting the data without tuning
- I did not attempt post-pruning because this project is already getting pretty long and I think the grader will agree I did enough already!

### **Bagging Models**

**Let's try bagging models to improve the performance of Logistic Regression, KNN, and Decision Tree**

#### Bagging Logistic Regression

In [ ]:
#Importing logistic regression model
from sklearn.linear_model import LogisticRegression
#importing the bagging classifier
from sklearn.ensemble import BaggingClassifier

bagging_lr=BaggingClassifier(base_estimator=LogisticRegression(solver='liblinear',random_state=1),random_state=1)
bagging_lr.fit(X_train_log_drop6,y_train_log) #I am using the X_train_drop6 because I previously determined these are the key features and I later realized this might cause issues with stacking, always next time!

In [ ]:
#Using above defined function to get accuracy, recall and precision on train and test set
bagging_lr_score=get_metrics_score(bagging_lr,X_train_log_drop6,X_test_log_drop6, y_train_log, y_test_log)

In [ ]:
make_confusion_matrix(bagging_lr,y_test_log,X_test_log_drop6)

- This seems to have offered an improvement in the Logistic Regressions ability to detect approvals or the majority class. Comes at a cost of reduced Recall or detection of the minority class.

#### Bagging KNN

In [ ]:
#Building the bagging classifier using kNN_s that I defined above.
bagging_kNN_s = BaggingClassifier(base_estimator=kNN_s,random_state=1)
bagging_kNN_s.fit(X_train_s,y_train_s)

In [ ]:
#Using above defined function to get accuracy, recall and precision on train and test set
bagging_kNN_s_score=get_metrics_score(bagging_kNN_s,X_train_s,X_test_s, y_train_s, y_test_s)

In [ ]:
make_confusion_matrix(bagging_kNN_s,y_test_s,X_test_s)

- Detecting alot of the majority, but with very poor recall and still overfitting.

In [ ]:
#building another bagging classifier but this time the model is fit to X_train_t and y_train_t
bagging_kNN_t = BaggingClassifier(base_estimator=kNN_t,random_state=1)
bagging_kNN_t.fit(X_train_t,y_train_t)

In [ ]:
#Using above defined function to get accuracy, recall and precision on train and test set
bagging_kNN_t_score=get_metrics_score(bagging_kNN_t,X_train_t,X_test_t, y_train_t, y_test_t)

In [ ]:
make_confusion_matrix(bagging_kNN_t,y_test_t,X_test_t)

- Without the bagging classifier, KNN was significantly improved by using the transformed variables. However by using the bagging classifier the improvement gained from transforming those variables is negligible.
- As a side note, KNN was not performing well and takes forever when used with bagging, so I will not use it any further in this project.

#### Bagging Decision Trees

In [ ]:
#base_estimator for bagging classifier is a decision tree by default
bagging_dtree_s=BaggingClassifier(random_state=1)
bagging_dtree_s.fit(X_train_s,y_train_s)

In [ ]:
#Using above defined function to get accuracy, recall and precision on train and test set
bagging_dtree_s_score=get_metrics_score(bagging_dtree_s,X_train_s,X_test_s, y_train_s, y_test_s)

In [ ]:
make_confusion_matrix(bagging_dtree_s,y_test_s,X_test_s)

- Not bad, but still overfitting considerably and poor recall. Let's try using transformed set.

In [ ]:
bagging_dtree_t = BaggingClassifier(random_state=1)
bagging_dtree_t.fit(X_train_t,y_train_t)

In [ ]:
#Using above defined function to get accuracy, recall and precision on train and test set
bagging_dtree_t_score=get_metrics_score(bagging_dtree_t,X_train_t,X_test_t, y_train_t, y_test_t)

In [ ]:
make_confusion_matrix(bagging_dtree_t,y_test_t,X_test_t)

- No significant improvment or decrease in performance. Lets try Random Forest Sampling instead.

#### Random Forest

In [ ]:
#importing the model
from sklearn.ensemble import RandomForestClassifier
#building the model with X_train_s
rf_s=RandomForestClassifier(random_state=1)
rf_s.fit(X_train_s,y_train_s)

In [ ]:
rf_s_score=get_metrics_score(rf_s,X_train_s,X_test_s,y_train_s,y_test_s)

In [ ]:
make_confusion_matrix(rf_s,y_test_s,X_test_s)

- Offers some improvements compared to the bagged decision trees. Still overfitting the data. Let's see what happens if we use the transformed data.

In [ ]:
#building the model with X_train_t
rf_t=RandomForestClassifier(random_state=1)
rf_t.fit(X_train_t,y_train_t)

In [ ]:
#Using above defined function to get accuracy, recall and precision on train and test set
rf_t_score=get_metrics_score(rf_t,X_train_t,X_test_t,y_train_t,y_test_t)

In [ ]:
make_confusion_matrix(rf_t,y_test_t,X_test_t)

- Again the result of the bagging method is an increased "generalization" such that the transformed variables have less impact on the final prediction.

### **Boosting Models**

#### AdaBoost

In [ ]:
# importing the model
from sklearn.ensemble import AdaBoostClassifier
# Building the model using X_train_s
adaBoost_s=AdaBoostClassifier(random_state=1)
adaBoost_s.fit(X_train_s,y_train_s)

In [ ]:
adaBoost_s_score=get_metrics_score(adaBoost_s,X_train_s,X_test_s,y_train_s,y_test_s)

In [ ]:
make_confusion_matrix(adaBoost_s,y_test_s,X_test_s)

- Wow this model is doing an excellent job of detecting the majority, but at the cost of reduced detection of the minority or poor recall scores. Does not seem to be overfitting.

In [ ]:
adaBoost_t = AdaBoostClassifier(random_state=1)
adaBoost_t.fit(X_train_t,y_train_t)

In [ ]:
#Using above defined function to get accuracy, recall and precision on train and test set
adaBoost_t_score=get_metrics_score(adaBoost_t,X_train_t,X_test_t,y_train_t,y_test_t)

In [ ]:
make_confusion_matrix(adaBoost_t,y_test_t,X_test_t)

- The generalization is pretty impressive. No difference at all when building it with the transformed set. In fact all the boosting models showed this behavior and I will avoid repeating it.

#### Gradient Boost

In [ ]:
#importing the model
from sklearn.ensemble import GradientBoostingClassifier
#building the model
gradBoost_s=GradientBoostingClassifier(random_state=1)
gradBoost_s.fit(X_train_s,y_train_s)

In [ ]:
gradBoost_s_score=get_metrics_score(gradBoost_s,X_train_s,X_test_s,y_train_s,y_test_s)

In [ ]:
make_confusion_matrix(gradBoost_s,y_test_s,X_test_s)

- Similar to performance of adaBoost, but it is "capturing" the minority class better with improved recall scores.

In [ ]:
gradBoost_t = GradientBoostingClassifier(random_state=1)
gradBoost_t.fit(X_train_t,y_train_t)

In [ ]:
#Using above defined function to get accuracy, recall and precision on train and test set
gradBoost_t_score=get_metrics_score(gradBoost_t,X_train_t,X_test_t,y_train_t,y_test_t)

In [ ]:
make_confusion_matrix(gradBoost_t,y_test_t,X_test_t)

#### XGradient Boost

In [ ]:
#importing the model
from xgboost import XGBClassifier
#building the model
eXtr_gradBoost_s = XGBClassifier(random_state=1)
eXtr_gradBoost_s.fit(X_train_s,y_train_s)

In [ ]:
eXtr_gradBoost_s_score=get_metrics_score(eXtr_gradBoost_s,X_train_s,X_test_s,y_train_s,y_test_s)

In [ ]:
make_confusion_matrix(eXtr_gradBoost_s,y_test_s,X_test_s)

- Not really much of an improvement. However it did help in some areas without overfitting, which is interesting.

In [ ]:
eXtr_gradBoost_t = XGBClassifier(random_state=1)
eXtr_gradBoost_t.fit(X_train_t,y_train_t)

In [ ]:
#Using above defined function to get accuracy, recall and precision on train and test set
eXtr_gradBoost_t_score=get_metrics_score(eXtr_gradBoost_t,X_train_t,X_test_t,y_train_t,y_test_t)

In [ ]:
make_confusion_matrix(eXtr_gradBoost_t,y_test_t,X_test_t)

##  Will tuning improve the model performance?

**From here on I only use the X_train_t or transformed set. In all cases, it performed the same or better than non-transformed X_train_s. Also it would take too long to tune two each of these models!**

### Tuning Logistic Regression, KNN, & Decision Tree

#### Logistic Regression Tuning

**ROC Curve**

In [ ]:
# I am essentially using the ROC curve to optimize the threshold by finding the maximum recall without excessively reducing specificity
# In other words, I want to detect the minority class

logit_roc_auc_train = metrics.roc_auc_score(y_train_log, lg_drop6.predict(X_train_log_drop6))
fpr, tpr, thresholds = metrics.roc_curve(y_train_log, lg_drop6.predict(X_train_log_drop6))
plt.figure(figsize=(7, 5))
plt.plot(fpr, tpr, label="Logistic Regression (area = %0.2f)" % logit_roc_auc_train)
plt.plot([0, 1], [0, 1], "r--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic")
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Optimal threshold as per AUC-ROC curve
# The optimal cut off would be where tpr is high and fpr is low
fpr, tpr, thresholds = metrics.roc_curve(y_train_log, lg_drop6.predict(X_train_log_drop6))

optimal_idx = np.argmax(tpr - fpr)
optimal_threshold_auc_roc = thresholds[optimal_idx]
print(optimal_threshold_auc_roc)

**Now we have the optimal threshold from ROC curve, lets check performance**

In [ ]:
log_reg_model_train_perf = model_performance_classification_statsmodels(
    lg_drop6, X_train_log_drop6, y_train_log,optimal_threshold_auc_roc
)

print("Training performance:")
log_reg_model_train_perf

In [ ]:
log_reg_model_test_perf = model_performance_classification_statsmodels(
    lg_drop6, X_test_log_drop6, y_test_log,optimal_threshold_auc_roc
)

print("Test performance:")
log_reg_model_test_perf

- Poor accuracy with decent recall

In [ ]:
# predicting on training set
# threshold is 0.3445, if predicted probability is greater than 0.3445 the observation will be classified as 1 or "Denied"

pred_test_logModel = lg_drop6.predict(X_test_log_drop6) > optimal_threshold_auc_roc
pred_test_logModel = np.round(pred_test_logModel)

cm = metrics.confusion_matrix(y_test_log, pred_test_logModel)
df_cm = pd.DataFrame(cm, index = [i for i in ["Actual - No","Actual - Yes"]],
              columns = [i for i in ['Predicted - No','Predicted - Yes']])
group_counts = ["{0:0.0f}".format(value) for value in
            cm.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                      cm.flatten()/np.sum(cm)]
labels = [f"{v1}\n{v2}" for v1, v2 in
          zip(group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=labels,fmt='')
plt.ylabel('True label')
plt.xlabel('Predicted label')

plt.show()

- This method appears to favor the minority class at the cost of the majority. I will try using the Precision-Recall curve to get a balance between detecting majority and minority class.

In [ ]:
y_scores = lg_drop6.predict(X_train_log_drop6)
prec, rec, tre = metrics.precision_recall_curve(y_train_log, y_scores,)


def plot_prec_recall_vs_tresh(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="precision")
    plt.plot(thresholds, recalls[:-1], "g--", label="recall")
    plt.xlabel("Threshold")
    plt.legend(loc="upper left")
    plt.ylim([0, 1])
    #plt.xlim([0.38,0.4])

plt.figure(figsize=(10, 7))
plot_prec_recall_vs_tresh(prec, rec, tre)
plt.show()

In [ ]:
# I find this by zooming in on the graph, this is not perfect, but adequate for this situation
optimal_threshold=0.39

**Now we have optimal threshold from precision-recall curve**

In [ ]:
log_reg_model_train_perf = model_performance_classification_statsmodels(
    lg_drop6, X_train_log_drop6, y_train_log,optimal_threshold
)

print("Training performance:")
log_reg_model_train_perf

In [ ]:
log_reg_model_test_perf = model_performance_classification_statsmodels(
    lg_drop6, X_test_log_drop6, y_test_log,optimal_threshold
)

print("Test performance:")
log_reg_model_test_perf

- Improved accuracy with a loss to recall and gains in precision. However the f1 score is actually lower than before, but not by much.

In [ ]:
# predicting on training set
# threshold is 0.39, if predicted probability is greater than this optimal_threshold the observation will be classified as 1 or "Denied"

pred_test_logModel = lg_drop6.predict(X_test_log_drop6) > optimal_threshold
pred_test_logModel = np.round(pred_test_logModel)

cm = metrics.confusion_matrix(y_test_log, pred_test_logModel)
df_cm = pd.DataFrame(cm, index = [i for i in ["Actual - No","Actual - Yes"]],
              columns = [i for i in ['Predicted - No','Predicted - Yes']])
group_counts = ["{0:0.0f}".format(value) for value in
            cm.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                      cm.flatten()/np.sum(cm)]
labels = [f"{v1}\n{v2}" for v1, v2 in
          zip(group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=labels,fmt='')
plt.ylabel('True label')
plt.xlabel('Predicted label')

plt.show()

- I prefer the second threshold gathered from Precision-Recall curve because it provides better overall accuracy. The goal is to shortlist applicants, so we would like to accurately detect as many approved applicants as possible! This means low false positives and false negatives are equally important.

- For now on, I will try to tune models based on f1-score and accuracy because I want to find the best balance between detection of minority and majority class. So that I can shortlist applicants and have minimal incorrectly shortlisted applicants.

#### KNN Tuning

In [ ]:
# Importing gridsearch for hyperparameter tuning
from sklearn.model_selection import GridSearchCV

kNN_t_hypertuned=KNeighborsClassifier()

# Grid of parameters to choose from
parameters = {
              "weights": ['uniform', 'distance',None],
              "n_neighbors": np.arange(3,10,1),
              "p": [1,2],
             }

# Type of scoring used to compare parameter combinations
acc_scorer = metrics.make_scorer(metrics.f1_score)

# Run the grid search
grid_obj = GridSearchCV(kNN_t_hypertuned, parameters, scoring=acc_scorer, cv=5,n_jobs=-1)
grid_obj = grid_obj.fit(X_train_t, y_train_t)

# Set the clf to the best combination of parameters
kNN_t_hypertuned = grid_obj.best_estimator_

# Fit the best algorithm to the data.
kNN_t_hypertuned.fit(X_train_t, y_train_t)

In [ ]:
#Using above defined function to get accuracy, recall and precision on train and test set
kNN_t_hypertuned_score=get_metrics_score(kNN_t_hypertuned,
                                         X_train_t,X_test_t,y_train_t,y_test_t)

In [ ]:
make_confusion_matrix(kNN_t_hypertuned,y_test_t,X_test_t)

- Model seems to have trouble detecting the minority class. Improved recall compared to original KNN model at the cost of a minor reduction in precision.


#### Decision Tree Tuning

In [ ]:
# To tune different models
from sklearn.model_selection import GridSearchCV

# Choose the type of classifier.
dtree_t_hypertuned = DecisionTreeClassifier(random_state=1)

# Grid of parameters to choose from
parameters = {
              "class_weight": [None, "balanced"],
              "max_depth": np.arange(2, 10, 1),
              "max_leaf_nodes": [10,30,50,80,100],
              "min_samples_split": [10, 30, 50, 70],
              "min_impurity_decrease": [0.001, 0.01, 0.1, 0.0]
             }

# Type of scoring used to compare parameter combinations
acc_scorer = metrics.make_scorer(metrics.f1_score)

# Run the grid search
grid_obj = GridSearchCV(dtree_t_hypertuned, parameters, scoring=acc_scorer, cv=5,n_jobs=-1)
grid_obj = grid_obj.fit(X_train_t, y_train_t)

# Set the clf to the best combination of parameters
dtree_t_hypertuned = grid_obj.best_estimator_

# Fit the best algorithm to the data.
dtree_t_hypertuned.fit(X_train_t, y_train_t)

In [ ]:
#Using above defined function to get accuracy, recall and precision on train and test set
dtree_t_hypertuned_score=get_metrics_score(dtree_t_hypertuned,X_train_t,X_test_t,y_train_t,y_test_t)

In [ ]:
make_confusion_matrix(dtree_t_hypertuned,y_test_t,X_test_t)

- Not overfitting. Similar performance to the tuned Logistic regression model.

In [ ]:
feature_names = X_train_t.columns
importances = dtree_t_hypertuned.feature_importances_
indices = np.argsort(importances)

plt.figure(figsize=(12,12))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='violet', align='center')
plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

- I focused more on model building in this project, but I do want to mention here that I can now confirm logistic regression and Decision trees both considered `education_of_employee`, `has_job_experience`, `continent_Europe`, and `unit_of_wage` to be significant features for prediction of visa status within this data.

### Tuning Bagging Models

#### Bagging Logistic Regression with tuning

In [ ]:
# Choose the type of classifier.
bagging_lr_hypertuned=BaggingClassifier(base_estimator=LogisticRegression(solver='liblinear',random_state=1),random_state=1)

# Grid of parameters to choose from
parameters = {
              'max_samples': [0.7,0.8,0.9,1],
              'max_features': [0.7,0.8,0.9,1],
              'n_estimators' : np.arange(80,120,10),
             }

# Type of scoring used to compare parameter combinations
acc_scorer = metrics.make_scorer(metrics.f1_score)

# Run the grid search
grid_obj = GridSearchCV(bagging_lr_hypertuned, parameters, scoring=acc_scorer,cv=5,n_jobs=-1)
grid_obj = grid_obj.fit(X_train_log_drop6, y_train_log)

# Set the clf to the best combination of parameters
bagging_lr_hypertuned = grid_obj.best_estimator_

# Fit the best algorithm to the data.
bagging_lr_hypertuned.fit(X_train_log_drop6,y_train_log)

In [ ]:
#Using above defined function to get accuracy, recall and precision on train and test set
bagging_lr_hypertuned_score=get_metrics_score(bagging_lr_hypertuned,X_train_log_drop6,X_test_log_drop6, y_train_log, y_test_log)

In [ ]:
make_confusion_matrix(bagging_lr_hypertuned,y_test_log,X_test_log_drop6)

- This model definitely provides an improvement to accuracy, but at the cost of reduced recall. Compared to the base model tuned with a Precision-Recall curve. Detecting 60% of the approved applicants accurately is not bad, perhaps stacking multiple models like this will help.

#### Bagging Trees with Tuning

In [ ]:
# Choose the type of classifier.
bagging_dtree_t_hypertuned = BaggingClassifier(random_state=1,n_jobs=-1)

# Grid of parameters to choose from
parameters = {
              'max_samples': [0.7,0.8,0.9,1],
              'max_features': [0.7,0.8,0.9,1],
              'n_estimators' : np.arange(80,120,10),
             }

# Type of scoring used to compare parameter combinations
acc_scorer = metrics.make_scorer(metrics.f1_score)

# Run the grid search
grid_obj = GridSearchCV(bagging_dtree_t_hypertuned, parameters, scoring=acc_scorer,cv=5,n_jobs=-1)
grid_obj = grid_obj.fit(X_train_t, y_train_t)

# Set the clf to the best combination of parameters
bagging_dtree_t_hypertuned = grid_obj.best_estimator_

# Fit the best algorithm to the data.
bagging_dtree_t_hypertuned.fit(X_train_t, y_train_t)

In [ ]:
#Using above defined function to get accuracy, recall and precision on train and test set
bagging_dtree_t_hypertuned_score=get_metrics_score(bagging_dtree_t_hypertuned,X_train_t,X_test_t,y_train_t,y_test_t)

In [ ]:
make_confusion_matrix(bagging_dtree_t_hypertuned,y_test_t,X_test_t)

- Okay, so compared to bagging the Logistic Regression models, better detection of the minority, but seems to be overfitting the data. I would rather use the previously tuned decision tree without bagging for stacking. Lets see if Random Forest can reduce the overfitting.

#### Random Forest with Tuning

In [ ]:
# Choose the type of classifier.
rf_t_hypertuned = RandomForestClassifier(random_state=1,n_jobs=-1)

# Grid of parameters to choose from
parameters = {
              "class_weight":['balanced'],
              "max_depth": [3,5,7],
              "n_estimators": np.arange(80,120,10),
              "max_features": [0.7,0.9,'log2','sqrt',None],
              "max_samples": [0.7,0.9,None],
             }

# Type of scoring used to compare parameter combinations
scorer = metrics.make_scorer(metrics.f1_score)

# Run the grid search
grid_obj = GridSearchCV(rf_t_hypertuned, parameters, scoring=scorer,cv=5,n_jobs=-1)
grid_obj = grid_obj.fit(X_train_t, y_train_t)

# Set the clf to the best combination of parameters
rf_t_hypertuned = grid_obj.best_estimator_

# Fit the best algorithm to the data.
rf_t_hypertuned.fit(X_train_t, y_train_t)

In [ ]:
rf_t_hypertuned_score=get_metrics_score(rf_t_hypertuned,X_train_t,X_test_t,y_train_t,y_test_t)

In [ ]:
make_confusion_matrix(rf_t_hypertuned,y_test_t,X_test_t)

- This model is doing an excellent job of accurately predicting the majority class without excessively ignoring the minority. Also, it is not overfitting like it was with Bagging alone.

In [ ]:
feature_names = X_train_t.columns
importances = rf_t_hypertuned.feature_importances_
indices = np.argsort(importances)

plt.figure(figsize=(12,12))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='violet', align='center')
plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

- This model seems to think that prevailing wage was important, which is different. Unfortunately this model seems to favor the minority class. Perhaps this is trying to tell me something I do not yet understand fully.

### Tuning Boosting Models

#### AdaBoost Tuning

In [ ]:
# Choose the type of classifier.
adaBoost_t_hypertuned = AdaBoostClassifier(random_state=1)

# Grid of parameters to choose from
parameters = {
              'n_estimators': np.arange(10,120,10),
              'learning_rate': [1, 0.1, 0.5, 0.01],
              }

# Type of scoring used to compare parameter combinations
scorer = metrics.make_scorer(metrics.f1_score)

# Run the grid search
grid_obj = GridSearchCV(adaBoost_t_hypertuned, parameters, scoring=scorer,cv=5,n_jobs=-1)
grid_obj = grid_obj.fit(X_train_t, y_train_t)

# Set the clf to the best combination of parameters
adaBoost_t_hypertuned = grid_obj.best_estimator_

# Fit the best algorithm to the data.
adaBoost_t_hypertuned.fit(X_train_t, y_train_t)

In [ ]:
adaBoost_t_hypertuned_score=get_metrics_score(adaBoost_t_hypertuned,X_train_t,X_test_t,y_train_t,y_test_t)

In [ ]:
make_confusion_matrix(adaBoost_t_hypertuned,y_test_t,X_test_t)

- There is no major sign of overfitting and the model is doing a great job of detecting approvals, but the minority class is being ignored.

In [ ]:
# Choose the type of classifier.
adaBoost_t_hypertuned_lr = AdaBoostClassifier(base_estimator=LogisticRegression(solver='liblinear',random_state=1),random_state=1)

# Grid of parameters to choose from
parameters = {
              'n_estimators': np.arange(10,120,10),
              'learning_rate': [1, 0.1, 0.5, 0.01],
              }

# Type of scoring used to compare parameter combinations
scorer = metrics.make_scorer(metrics.f1_score)

# Run the grid search
grid_obj = GridSearchCV(adaBoost_t_hypertuned_lr, parameters, scoring=scorer,cv=5,n_jobs=-1)
grid_obj = grid_obj.fit(X_train_log_drop6, y_train_log)

# Set the clf to the best combination of parameters
adaBoost_t_hypertuned_lr = grid_obj.best_estimator_

# Fit the best algorithm to the data.
adaBoost_t_hypertuned_lr.fit(X_train_log_drop6, y_train_log)

In [ ]:
adaBoost_t_hypertuned_lr_score=get_metrics_score(adaBoost_t_hypertuned_lr,X_train_log_drop6,X_test_log_drop6,y_train_log,y_test_log)

In [ ]:
make_confusion_matrix(adaBoost_t_hypertuned_lr,y_test_log,X_test_log_drop6)

- I see some slight improvements in performance with an increased tendency to ignore the minority class compared to using decison tree as a base estimator.  

#### Gradient Boost Tuning

In [ ]:
# Choose the type of classifier.
grad_Boost_t_hypertuned = GradientBoostingClassifier(random_state=1)

# Grid of parameters to choose from
parameters = {
              'n_estimators': np.arange(50,125,25),
              'subsample':[0.7,0.8,0.9,1],
              'max_features':[0.7,0.8,0.9,1],
              'max_depth':[3,5,7,10]
              }

# Type of scoring used to compare parameter combinations
scorer = metrics.make_scorer(metrics.f1_score)

# Run the grid search
grid_obj = GridSearchCV(grad_Boost_t_hypertuned, parameters, scoring=scorer,cv=5,n_jobs=-1)
grid_obj = grid_obj.fit(X_train_t, y_train_t)

# Set the clf to the best combination of parameters
grad_Boost_t_hypertuned = grid_obj.best_estimator_

# Fit the best algorithm to the data.
grad_Boost_t_hypertuned.fit(X_train_t, y_train_t)

In [ ]:
grad_Boost_t_hypertuned_score=get_metrics_score(grad_Boost_t_hypertuned,X_train_t,X_test_t,y_train_t,y_test_t)

In [ ]:
make_confusion_matrix(grad_Boost_t_hypertuned,y_test_t,X_test_t)

- Decent performance with signs of overfitting. Maybe using XGBoost will help improve the performance.

In [ ]:
feature_names = X_train_t.columns
importances = grad_Boost_t_hypertuned.feature_importances_
indices = np.argsort(importances)

plt.figure(figsize=(12,12))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='violet', align='center')
plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

#### XGradient Boost Tuning

In [ ]:
from xgboost import XGBClassifier

# Choose the type of classifier.
eXtr_gradBoost_t_hypertuned = XGBClassifier(random_state=1, verbosity = 0)

# Grid of parameters to choose from
parameters = {
              'n_estimators': np.arange(50,125,25),
              'subsample':[0.7, 0.8, 0.9, 1],
              'gamma':[0, 1, 3, 5],
              'colsample_bytree':[0.7, 0.8, 0.9, 1],
              'colsample_bylevel':[0.7, 0.8, 0.9, 1]
              }

# Type of scoring used to compare parameter combinations
scorer = metrics.make_scorer(metrics.f1_score)

# Run the grid search
grid_obj = GridSearchCV(eXtr_gradBoost_t_hypertuned, parameters, scoring=scorer,cv=5,n_jobs=-1)
grid_obj = grid_obj.fit(X_train_t, y_train_t)

# Set the clf to the best combination of parameters
eXtr_gradBoost_t_hypertuned = grid_obj.best_estimator_

# Fit the best algorithm to the data.
eXtr_gradBoost_t_hypertuned.fit(X_train_t, y_train_t)

In [ ]:
eXtr_gradBoost_t_hypertuned_score=get_metrics_score(eXtr_gradBoost_t_hypertuned,X_train_t,X_test_t,y_train_t,y_test_t)

In [ ]:
make_confusion_matrix(eXtr_gradBoost_t_hypertuned,y_test_t,X_test_t)

- It is definitely an improvement over Gradient Boosting. The overfitting problem appears to have been solved and the performance improved! I found this to be actually pretty impressive. Even the f1 score is not showing signs of overfit.
- In the final stacking model I should use this as my final estimator. Basically it seems like it has respectable accuracy without sacrificing too much recall.

In [ ]:
feature_names = X_train_t.columns
importances = eXtr_gradBoost_t_hypertuned.feature_importances_
indices = np.argsort(importances)

plt.figure(figsize=(12,12))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='violet', align='center')
plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

- Interesting, it has decided to focus more on 'unit_of_wage_hour' than 'education_of_employee'

### Stacking Models

In [ ]:
estimators=[('Decision Tree', dtree_t_hypertuned),('Random Forest', rf_t_hypertuned),
           ('Gradient Boosting', grad_Boost_t_hypertuned)]
final_estimator=XGBClassifier(random_state=1)

In [ ]:
#import the library for stacking model
from sklearn.ensemble import StackingClassifier

stacking_estimator=StackingClassifier(estimators=estimators, final_estimator=final_estimator,cv=5)
stacking_estimator.fit(X_train_t,y_train_t)

In [ ]:
stacked_score=get_metrics_score(stacking_estimator,X_train_t,X_test_t,y_train_t,y_test_t)

In [ ]:
make_confusion_matrix(stacking_estimator,y_test_t,X_test_t)

- Performance very comparible to tuned gradient boost with a major improvement to overfitting, particularly for the f1 score and recall, which are both have proven to be very important scores.

## Model Performance Comparison and Conclusions

In [ ]:
# I print this out seperate because I had to set thresholds and this was my beginner way of solving that problem
print("Train/Test performance:")
log_reg_model_train_perf,log_reg_model_test_perf

In [ ]:
# Before tuning performance comparison

index=["Accuracy-Train","Accuracy-Test","Recall -Train","Recall -Test","Precision-Train","Precision-Test","F1-Train","F1-Test"]

models_train_comp_df = pd.concat(
    [dtree_t_weighted_score, kNN_t_score,bagging_lr_score,bagging_kNN_t_score,
    bagging_dtree_t_score,rf_t_score,adaBoost_t_score,gradBoost_t_score,
    eXtr_gradBoost_t_score],
    axis=1,
)

models_train_comp_df.columns = [
    "Decision Tree Weighted",
    "KNN",
    "Bagging Logistic Reg",
    "Bagging KNN",
    "Bagging Decision Tree",
    "Random Forest",
    "AdaBoost",
    "Gradient Boost",
    "XGB"]

models_train_comp_df.index=index

print("Training performance comparison:")
models_train_comp_df

In [ ]:
# After tuning performance comparison

index=["Accuracy-Train","Accuracy-Test","Recall -Train","Recall -Test","Precision-Train","Precision-Test""Precision-Test","F1-Train","F1-Test"]

models_train_comp_df = pd.concat(
    [dtree_t_hypertuned_score, kNN_t_hypertuned_score,bagging_lr_hypertuned_score,bagging_dtree_t_hypertuned_score,
    rf_t_hypertuned_score,adaBoost_t_hypertuned_score,grad_Boost_t_hypertuned_score,eXtr_gradBoost_t_hypertuned_score,
    stacked_score],
    axis=1,
)

models_train_comp_df.columns = [
    "Decision Tree Tuned",
    "KNN Tuned",
    "Baggin Logistic Reg Tuned",
    "Bagging Decision Tree Tuned",
    "Random Forest Tuned",
    "AdaBoost Tuned",
    "Gradient Boost Tuned",
    "XGB Tuned",
    "Stacking Tuned"]
models_train_comp_df.index=index

print("Tuned performance comparison:")
models_train_comp_df

## Actionable Insights and Recommendations

Insights:
  - Specifically, `education_of_employee`, `has_job_experience`, `continent_Europe`, and `unit_of_wage` were indicated as important features by the analysis.
      - This was further backed up by the EDA. For example:
        - "Europe" was the most likely to be "Certified".
        - "Hourly" was the most likely to be "Denied" and so on.
  - The stacking model appears to provide the best performance and offers the benefit of enhanced generalization. However the models were not perfect and some errors will be made in shortlisting candidates.<br></br>

Recommendations:
- I recommend using the stacked model to shortlist applicants, but then utilize a secondary screen to catch mistakes made in shortlisting applicants.
  - The secondary screen can be a streamlined version of the original visa application process.
    - Perhaps it can take advantage of the primary screen to speed up the process. For instance, a shortlisted candidate that has a Doctorate with job experience from Europe and with a yearly salary could be expedited.
- Of all the features, look out for yearly salaried employees, highly educated individuals, and especially if they have job experience because they are good candidates for shortlisting without adding alot of error.


In [ ]:
%%shell
jupyter nbconvert --to html /content/DSBA_Project_ET_EasyVisa_Fullcode.ipynb